In [29]:
import os
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

tmpdir = tempfile.mkdtemp()


In [30]:
print(tmpdir)

/tmp/tmpxg0seqor


In [31]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

# 1. pretrained_model

In [32]:
pretrained_model = tf.keras.applications.MobileNet()
print(pretrained_model)

In [33]:
mobilenet_save_path = os.path.join(tmpdir, "mobilenet/1/")
tf.saved_model.save(pretrained_model, mobilenet_save_path)

mobilenet_keras_save_path = os.path.join(tmpdir, "mobilenet_keras/1/")
tf.keras.models.save_model(pretrained_model, mobilenet_keras_save_path)


In [34]:
pip install seedir


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import seedir as sd
sd.seedir(mobilenet_save_path)

1/
├─variables/
│ ├─variables.index
│ └─variables.data-00000-of-00001
├─assets/
├─saved_model.pb
└─fingerprint.pb


In [36]:
loaded = tf.saved_model.load(mobilenet_save_path)
print(loaded)
print(loaded.signatures)
print(list(loaded.signatures.keys()))  # ["serving_default"]

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fcda665e490>
_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input_2) at 0x7FCDA58E9AC0>})
['serving_default']


# 2. custom model

In [37]:
class CustomModule(tf.Module):

  def __init__(self):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function
  def __call__(self, x):
    print('Tracing with', x)
    return x * self.v

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def mutate(self, new_v):
    self.v.assign(new_v)

module = CustomModule()

In [38]:
module_no_signatures_path = os.path.join(tmpdir, 'module_no_signatures')
module(tf.constant(0.))
print('Saving model...')
tf.saved_model.save(module, module_no_signatures_path)

Tracing with Tensor("x:0", shape=(), dtype=float32)
Saving model...
Tracing with Tensor("x:0", shape=(), dtype=float32)


In [39]:
sd.seedir(module_no_signatures_path)

module_no_signatures/
├─variables/
│ ├─variables.index
│ └─variables.data-00000-of-00001
├─assets/
├─saved_model.pb
└─fingerprint.pb


In [40]:
loaded = tf.saved_model.load(module_no_signatures_path)
print(loaded)
print(loaded.signatures)
print(list(loaded.signatures.keys()))

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fcda6dfef70>
_SignatureMap({})
[]


In [41]:
#asign a signature to info the model/function input and output 
module_with_signature_path = os.path.join(tmpdir, 'module_with_signature')
call = module.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))
tf.saved_model.save(module, module_with_signature_path, signatures=call)

Tracing with Tensor("x:0", dtype=float32)
Tracing with Tensor("x:0", dtype=float32)


In [42]:
sd.seedir(module_with_signature_path)

module_with_signature/
├─variables/
│ ├─variables.index
│ └─variables.data-00000-of-00001
├─assets/
├─saved_model.pb
└─fingerprint.pb


# 3. keras model

In [43]:
keras_model_save_path = os.path.join(tmpdir, 'keras_savedmodel')
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, input_shape=(3,)),
    tf.keras.layers.Softmax()])
model.save(keras_model_save_path)

In [44]:
loaded_model = tf.keras.models.load_model(keras_model_save_path)

In [45]:
print(loaded_model)
print(loaded_model)
print(loaded_model.signatures)
print(list(loaded_model.signatures.keys()))

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, dense_2_input) at 0x7FCDA6620970>})
['serving_default']


In [46]:
# test use saved_model to load keras model, it works
loaded = tf.saved_model.load(keras_model_save_path)
print(loaded)
print(loaded.signatures)
print(list(loaded.signatures.keys()))

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fcda60a04c0>
_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, dense_2_input) at 0x7FCDA55A6C40>})
['serving_default']


In [47]:
# test use keras.models.load_model load no-keras model
loaded_model = tf.keras.models.load_model(module_with_signature_path)

In [48]:
print(loaded_model)
print(loaded_model)
print(loaded_model.signatures)
print(list(loaded_model.signatures.keys()))

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fcda59c6cd0>
<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fcda59c6cd0>
_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, x) at 0x7FCDA666BA30>})
['serving_default']


1. tf.keras.models.load_model load kera_model(savedModel) -> keras.engine.sequential.Sequential object
2. tf.saved_model.load load custom_model(savedModel) -> tensorflow.python.saved_model.load.Loader._recreate_base_user_object
3. tf.saved_model.load load kera_model -> tensorflow.python.saved_model.load.Loader._recreate_base_user_object
4. tf.keras.models.load_model load custom_model(savedModel) -> tensorflow.python.saved_model.load.Loader._recreate_base_user_object



### keras H5

The default and recommended format to use is the TensorFlow SavedModel format. In TensorFlow 2.0 and higher, you can just do: model.save(your_file_path).

For explicitness, you can also use model.save(your_file_path, save_format='tf').

Keras still supports its original HDF5-based saving format. To save a model in HDF5 format, use model.save(your_file_path, save_format='h5')

In [49]:
keras_modelh5_save_path = os.path.join(tmpdir, 'keras_h5model.h5')
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, input_shape=(3,)),
    tf.keras.layers.Softmax()])
model.save(keras_modelh5_save_path, save_format='h5')

# 4. TensorFlow.js Converter


1. tf_saved_model  ->  tfjs_graph_model



2. keras_saved_model  -> tfjs_graph_model or tfjs_layers_model


5. Test tf.keras.models.save_model custom module

In [51]:
class CustomModule(tf.Module):

  def __init__(self):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function
  def __call__(self, x):
    print('Tracing with', x)
    return x * self.v

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def mutate(self, new_v):
    self.v.assign(new_v)

# module = CustomModule()
# keras_savedModel_CustomModule_path = os.path.join(tmpdir, 'keras_savedModel_CustomModule')
# tf.keras.models.save_model(module, keras_savedModel_CustomModule_path)


# Module have no graph ?

In [52]:
class CustomModule(tf.Module):

  def __init__(self):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function
  def __call__(self, x):
    print('Tracing with', x)
    return x * self.v * 2 * x

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def mutate(self, new_v):
    self.v.assign(new_v)

module = CustomModule()

In [53]:
module_no_signatures_no_graph_path = os.path.join(tmpdir, 'module_no_signatures_no_graph')
module(tf.constant(0.))
print('Saving model...')
tf.saved_model.save(module, module_no_signatures_no_graph_path)

Tracing with Tensor("x:0", shape=(), dtype=float32)
Saving model...
Tracing with Tensor("x:0", shape=(), dtype=float32)


In [54]:
print(module)

In [ ]:
!pwd
!zip -r colab.zip $tmpdir